# Neural Network for choosing stock

The following are some ML models (not entirely NN) trained to choose the best stocks, from which we can form our portfolio.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
# from statsmodels.tsa.arima.model import ARIMA

ModuleNotFoundError: No module named 'statsmodels'

## Deep Multilayer Perceptron (DMLP)

In [ ]:
class DMLPModel(nn.Module):
    def __init__(self, input_dim=60, hidden_dim=15, num_hidden_layers=6, output_dim=1):
        super(DMLPModel, self).__init__()
        self.hidden_layers = nn.ModuleList()
        
        self.hidden_layers.append(nn.Linear(input_dim, hidden_dim))
        
        for _ in range(num_hidden_layers - 1):
            self.hidden_layers.append(nn.Linear(hidden_dim, hidden_dim))

        self.output_layer = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        for layer in self.hidden_layers:
            x = self.relu(layer(x))
        x = self.output_layer(x)
        return x

## Long short-term memory (LSTM)

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim=60, hidden_dim=5, num_layers=4, output_dim=1, dropout_rate=0.4, recurrent_dropout_rate=0.3):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=recurrent_dropout_rate)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)
    
    def forward(self, x):
        x = x.unsqueeze(1)
        out, _ = self.lstm(x)
        out = self.dropout(out[:, -1, :])  
        out = self.fc(out)
        return out

## Convolutional Neural Network (CNN)

In [ ]:
class CNNModel(nn.Module):
    def __init__(self, input_dim=60, output_dim=1, num_filters=2, kernel_size=2, hidden_dim=2):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=num_filters, kernel_size=kernel_size)
        self.pool = nn.MaxPool1d(kernel_size=kernel_size)
        self.fc1 = nn.Linear(num_filters * (input_dim // 2 - 1), hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = x.unsqueeze(1)  # add channel
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

## Random Forest

In [ ]:
def RandomForestModel(n_estimators=500, max_depth=20,min_samples_split=10,min_samples_leaf=10,max_features=40):
    return RandomForestRegressor(
                n_estimators=n_estimators,
                max_depth=max_depth,
                min_samples_split=min_samples_split,
                min_samples_leaf=min_samples_leaf,
                max_features=max_features
                )

## SVR

In [ ]:
def SVRModel(C=2**2, gamma=2**-3):
    return SVR(kernel='rbf', C=C, gamma=gamma)

In [ ]:
rnn = LSTMModel()
DMLP = DMLPModel()
CNN = CNNModel()
RF = RandomForestModel()
SVR = SVRModel()

In [ ]:
# Sanity check
input = torch.randn(5, 60)
train = torch.randn(5,60)
label = torch.randn(5,1)
SVR.fit(train, label)
output = SVR.predict(input)
print(output)